In [1]:
import coffea, hist
print(coffea.__version__)
print(hist.__version__)

2025.10.2
2.9.0


In [2]:
import awkward as ak
import numpy as np
import hist

#import boost_histogram as bh

#import dask.array as da

#import dask_histogram as dh

import json
import sys

from coffea import processor
from coffea.nanoevents import NanoAODSchema, BaseSchema
from pathlib import Path

current_dir = Path.cwd()

src_dir = current_dir.parent.parent / "src"
sys.path.append(str(src_dir))

from selections import get_event_combos, make_dr_hist, make_dr_div_mll_hist
from jet_trigger_mask import get_trigger_mask
import lep_tagger_UL as tagger_UL
import lep_tagger as tagger_preUL

class Processor(processor.ProcessorABC):
    def __init__(self, mode="virtual"):
        assert mode in ["virtual", "eager", "dask"]
        self._mode = mode

    def process(self, events):
        dataset = events.metadata["dataset"]
        print(dataset)

        is_TChiWZ = "TChiWZ" in dataset or "SMS-TChiWZ" in dataset
        is_mc = events.metadata.get("is_mc", False)
        is_ntuple = events.metadata.get("is_ntuple", False)

        is_UL = events.metadata.get("is_UL", False)

        def concat_leps(my_events, is_UL):

            tagger = tagger_UL if is_UL else tagger_preUL
            tag_qual = tagger.tag_qual
            
            ele = tag_qual(my_events.Electron, 'ele')
            mu = tag_qual(my_events.Muon, 'mu')
            return ak.with_name(ak.concatenate([ele, mu], axis=1), "PtEtaPhiMCandidate")
        
        output = {} #instantiate results here, fill later

        def good_PV(events):
            return (
                (events.PV.chi2 >= 0) &
                (events.PV.ndof >= 5) &
                (np.abs(events.PV.z) <= 24) &
                ((events.PV.x**2 + events.PV.y**2) <= 4)
            )

        

        if not is_ntuple:
            mp_300_290_events = events[events.GenModel.TChiWZ_ZToLL_300_290]

            leps = concat_leps(mp_300_290_events, is_UL)
            baseline_leps = leps[leps.isBaseline]
    
            cut_events_1 = mp_300_290_events[
                (mp_300_290_events.MET.pt > 150)
                ]
    
            cut_events_2 = mp_300_290_events[
                (mp_300_290_events.MET.pt > 150) &
                (ak.num(mp_300_290_events.Jet) > 0)
                ]
    
            cut_events_3 = mp_300_290_events[
                (mp_300_290_events.MET.pt > 150)   &
                (ak.num(mp_300_290_events.Jet) > 0) &
                ((ak.num(mp_300_290_events.Electron) + ak.num(mp_300_290_events.Muon)) >= 2)
                ]

            cut_events_3_2 = mp_300_290_events[
                (mp_300_290_events.MET.pt > 150)   &
                (ak.num(mp_300_290_events.Jet) > 0) &
                ((ak.num(mp_300_290_events.Electron) + ak.num(mp_300_290_events.Muon)) >= 2) &
                good_PV(mp_300_290_events)
                ]

            cut_events_3_3 = mp_300_290_events[
                (mp_300_290_events.MET.pt > 150)   &
                (ak.num(mp_300_290_events.Jet) > 0) &
                ((ak.num(mp_300_290_events.Electron) + ak.num(mp_300_290_events.Muon)) >= 2) &
                good_PV(mp_300_290_events) &
                (ak.num(baseline_leps) == 2)
                ]

            cut_events_3_4 = mp_300_290_events[
                (mp_300_290_events.MET.pt > 150)   &
                (ak.num(mp_300_290_events.Jet) > 0) &
                ((ak.num(mp_300_290_events.Electron) + ak.num(mp_300_290_events.Muon)) >= 2) &
                good_PV(mp_300_290_events) &
                (ak.num(baseline_leps) >= 2)
                ]

            cut_events_4 = mp_300_290_events[
                (mp_300_290_events.MET.pt > 150)   &
                (ak.num(mp_300_290_events.Jet) > 0) &
                ((ak.num(mp_300_290_events.Electron) + ak.num(mp_300_290_events.Muon)) == 2)
                ]

            cut_events_5 = mp_300_290_events[
                (mp_300_290_events.MET.pt > 150)   &
                (ak.num(mp_300_290_events.Jet) > 0) &
                ((ak.num(mp_300_290_events.Electron) + ak.num(mp_300_290_events.Muon)) == 2) &
                (ak.num(baseline_leps) == 2)
                ]
            
            cut_events_6 = mp_300_290_events[
                (mp_300_290_events.MET.pt > 150)   &
                (ak.num(mp_300_290_events.Jet) > 0) &
                (ak.num(baseline_leps) == 2)
                ]

            

            #Next apply baseline events, could do it from the ground up like baseline_leps = leps[leps.isBaseline]

            output["events_300_290"] = len(mp_300_290_events)
            output["cut_events_1"] = len(cut_events_1)
            output["cut_events_2"] = len(cut_events_2)
            output["cut_events_3"] = len(cut_events_3)
            output["cut_events_3_2"] = len(cut_events_3_2)
            output["cut_events_3_3"] = len(cut_events_3_3)
            output["cut_events_3_4"] = len(cut_events_3_4)
            output["cut_events_4"] = len(cut_events_4)
            output["cut_events_5"] = len(cut_events_5)
            output["cut_events_6"] = len(cut_events_6)
            
            
        if is_ntuple:
            output["events"] = len(events)
            output["dilep"] = len(events.Nlep >= 2)
            output["dilep_2"] = len(events.Nlep == 2)
            
        """
        def TChiWZ_mass_points(events):
            # Split by mass points
            events_300_290 = events[events.GenModel.TChiWZ_ZToLL_300_290]
            events_300_295 = events[events.GenModel.TChiWZ_ZToLL_300_295]
            events_300_297 = events[events.GenModel.TChiWZ_ZToLL_300_297]

            mp_events_list = [
                ("300_290", events_300_290),
                ("300_295", events_300_295),
                ("300_297", events_300_297)
            ] 
            
            return mp_events_list
            
        if is_TChiWZ:

            mp_event_list = TChiWZ_mass_points(events)

            for mp_name, mp_events in mp_event_list:

                output[mp_name] = {}

                output[mp_name]["events"] = len(events)
                
                cut_all_events(
                mp_events[(mp_events.MET.pt <= 150) & (ak.num(mp_events.Jet) > 0) & ((ak.num(mp_events.Electron) + ak.num(mp_events.Muon)) >= 2)],
                output[mp_name],
                f"{mp_name}_MET_lt_150"
            )
                cut_all_events(
                    mp_events[(mp_events.MET.pt > 150) & (ak.num(mp_events.Jet) > 0) & ((ak.num(mp_events.Electron) + ak.num(mp_events.Muon)) >= 2)],
                    output[mp_name],
                    f"{mp_name}_MET_gt_150"
                )
            
                cut_all_events(
                    mp_events[(ak.num(mp_events.Jet) > 0) & ((ak.num(mp_events.Electron) + ak.num(mp_events.Muon)) >= 2)],
                    output[mp_name],
                    mp_name
                )

        else:
            output["events"] = len(events)
            cut_all_events(
                events[(events.MET.pt <= 150) & get_trigger_mask(events, is_mc, is_UL) & (ak.num(events.Jet) > 0) & ((ak.num(events.Electron) + ak.num(events.Muon)) >= 2)],
                output,
                "MET_lt_150"
            )
            
            if is_mc:
                cut_all_events(
                    events[(events.MET.pt > 150) & get_trigger_mask(events, is_mc, is_UL) & (ak.num(events.Jet) > 0) & ((ak.num(events.Electron) + ak.num(events.Muon)) >= 2)],
                    output,
                    "MET_gt_150"
                )
        """
                
        return output  
        
    def postprocess(self, accumulator):
        pass

In [3]:
import cloudpickle
import os
from datetime import datetime
from dask.distributed import Client

# Create pickles directory if it doesn't exist
os.makedirs("pikls", exist_ok=True)

# Set mode
mode = "dask"  # or "dask"
make_pikl = False
#fileset_name = "fileset_teeny.txt"
#fileset_name = "fileset_nano.txt"
fileset_name = "fileset_TChiWZ_2018_naod.txt"
#fileset_name = "fileset_TChiWZ_2018_ntuple.txt"
#fileset_name = "fileset_ntuples.txt"
#fileset_name = "fileset.txt"

results = {}
all_metrics = {}

with open(fileset_name, "r") as f:
    exec(f.read())

# Set up executor based on mode
if mode == "dask":
    client = Client("tls://localhost:8786")
    #client.restart() # may be needed at times to refresh the files uploaded below
    executor = processor.DaskExecutor(client=client)
    client.upload_file("jet_trigger_mask.py", load=False)
    client.upload_file("selections.py", load=False)
    client.upload_file("lep_tagger.py", load=False)
    client.upload_file("lep_tagger_UL.py", load=False)
    client.upload_file("vid_unpacked.py", load=False)
    client.upload_file("gen_tagger.py", load=False)
    client.upload_file("gen_filter.py", load=False)
else:  # virtual mode
    executor = processor.IterativeExecutor()

# Set up output directory
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_dir = f"pikls/{timestamp}"
os.makedirs(output_dir, exist_ok=True)

# Process each dataset
for dataset_name in fileset.keys():
    print(f"\nProcessing {dataset_name}...")
    
    single_fileset = {dataset_name: fileset[dataset_name]}
    
    runner = processor.Runner(
        executor=executor,
        schema=NanoAODSchema,
        savemetrics=True,
        skipbadfiles=False,
    )
    
    result, metrics = runner(single_fileset, processor_instance=Processor(mode=mode))
    results[dataset_name] = result
    all_metrics[dataset_name] = metrics
    
    if make_pikl:
        output_file = f"{output_dir}/{dataset_name}.pkl"
        
        with open(output_file, "wb") as f:
            cloudpickle.dump({'result': result, 'metrics': metrics}, f)
        
        print(f"Saved {output_file}")

# Save combined results
if make_pikl:
    output_file = f"{output_dir}/all_datasets_full.pkl"
    
    with open(output_file, "wb") as f:
        cloudpickle.dump({'results': results, 'metrics': all_metrics}, f)
    
    print(f"Saved {output_file}")


Processing SMS-TChiWZZToLLmZMin-0p1TuneCP513TeV-madgraphMLM-pythia8RunIISummer20UL18NanoAODv9-106Xupgrade2018realisticv16L1v1-v1NANOAODSIM...


Output()

Output()

MET LESS THAN 150 (NOT IN SKIM)

mp_300_290_events = events[events.GenModel.TChiWZ_ZToLL_300_290]
    
            cut_events_1 = mp_300_290_events[
                (mp_300_290_events.MET.pt <= 150)
                ]
    
            cut_events_2 = mp_300_290_events[
                (mp_300_290_events.MET.pt <= 150) &
                (ak.num(mp_300_290_events.Jet) > 0)
                ]
    
            cut_events_3 = mp_300_290_events[
                (mp_300_290_events.MET.pt <= 150)   &
                (ak.num(mp_300_290_events.Jet) > 0) &
                ((ak.num(mp_300_290_events.Electron) + ak.num(mp_300_290_events.Muon)) >= 2)
                ]

            cut_events_4 = mp_300_290_events[
                (mp_300_290_events.MET.pt <= 150)   &
                (ak.num(mp_300_290_events.Jet) > 0) &
                ((ak.num(mp_300_290_events.Electron) + ak.num(mp_300_290_events.Muon)) == 2)
                ]

            cut_events_5 = mp_300_290_events[
                (mp_300_290_events.MET.pt <= 150)   &
                (ak.num(mp_300_290_events.Jet) > 0) &
                ((ak.num(mp_300_290_events.Electron) + ak.num(mp_300_290_events.Muon)) == 2) &
                (ak.num(baseline_leps) == 2)
                ]
            
            cut_events_6 = mp_300_290_events[
                (mp_300_290_events.MET.pt <= 150)   &
                (ak.num(mp_300_290_events.Jet) > 0) &
                (ak.num(baseline_leps) == 2)
                ]

{'events_300_290': 156543,

 'cut_events_1': 139751,
 
 'cut_events_2': 131980,
 
 'cut_events_3': 45319,
 
 'cut_events_4': 32977,
 
 'cut_events_5': 1669,
 
 'cut_events_6': 2932
 }

# MET GREATER THAN 150

 cut_events_1 = mp_300_290_events[
                (mp_300_290_events.MET.pt > 150)
                ]
    
            cut_events_2 = mp_300_290_events[
                (mp_300_290_events.MET.pt > 150) &
                (ak.num(mp_300_290_events.Jet) > 0)
                ]
    
            cut_events_3 = mp_300_290_events[
                (mp_300_290_events.MET.pt > 150)   &
                (ak.num(mp_300_290_events.Jet) > 0) &
                ((ak.num(mp_300_290_events.Electron) + ak.num(mp_300_290_events.Muon)) >= 2)
                ]

            cut_events_4 = mp_300_290_events[
                (mp_300_290_events.MET.pt > 150)   &
                (ak.num(mp_300_290_events.Jet) > 0) &
                ((ak.num(mp_300_290_events.Electron) + ak.num(mp_300_290_events.Muon)) == 2)
                ]

            cut_events_5 = mp_300_290_events[
                (mp_300_290_events.MET.pt > 150)   &
                (ak.num(mp_300_290_events.Jet) > 0) &
                ((ak.num(mp_300_290_events.Electron) + ak.num(mp_300_290_events.Muon)) == 2) &
                (ak.num(baseline_leps) == 2)
                ]
            
            cut_events_6 = mp_300_290_events[
                (mp_300_290_events.MET.pt > 150)   &
                (ak.num(mp_300_290_events.Jet) > 0) &
                (ak.num(baseline_leps) == 2)


                {'events_300_290': 156543,
                
 'cut_events_1': 16792,
 
 'cut_events_2': 16792,
 
 'cut_events_3': 8830,
 
 'cut_events_4': 4874,
 
 'cut_events_5': 235,
 
 'cut_events_6': 637}

for comparison:

{'events': 5752,  NLep ≥ 2 events in ntuple 2018 TChiWZ 300_290

 'dilep_events': 4600,  NLep == 2 events in "" 

# Conclusion:

skim events NLep == 2: 4600

my cut_events_3 : 8830

These two numbers should be the same, however Zach's skim has possibly 4 (?) more cuts on it than I do. Claude found in `ReducedNtuple.cc`:

| Line(s) | Cut Description | Applied To | Purpose |
|---------|----------------|------------|---------|
| 870-872 | Good event filter | Data only | Data quality |
| 877-878 | Good primary vertex | All events | Vertex quality |
| 891 | MET not NaN | All events | Data integrity |
| 892-893 | MET ≥ 0 | All events | Sanity check |
| **1021** | **≥2 leptons AND MET>150 AND ≥1 jet** | **All events** | **Main selection** |
| 1026 | MN1/MP < 0.45 rejection | Signal MC only | Compressed spectrum |

So I'm missing the:
- "Good PV" cut,
- MET not NaN (idk if this applies to my framework, coffea may already count those as rejected in the array masking?)
- "compressed spectrum" cut, MN1/MP < 0.45 (doesn't matter for me with 300/290)


I added the Good PV cut:

```
def good_PV(events):
            return (
                (events.PV.chi2 >= 0) &
                (events.PV.ndof >= 5) &
                (np.abs(events.PV.z) <= 24) &
                ((events.PV.x**2 + events.PV.y**2) <= 4)
            )

cut_events_3_2 = mp_300_290_events[
                (mp_300_290_events.MET.pt > 150)   &
                (ak.num(mp_300_290_events.Jet) > 0) &
                ((ak.num(mp_300_290_events.Electron) + ak.num(mp_300_290_events.Muon)) >= 2) &
                good_PV(mp_300_290_events)
                ]

```

{'events_300_290': 156543,
 
 'cut_events_1': 16792,
 
 'cut_events_2': 16792,
 
 'cut_events_3': 8830,
 
 'cut_events_3_2': 8821,
 
 'cut_events_4': 4874,
 
 'cut_events_5': 235,
 
 'cut_events_6': 637,

In [4]:
result

{'events_300_290': 156543,
 'cut_events_1': 16792,
 'cut_events_2': 16792,
 'cut_events_3': 8830,
 'cut_events_3_2': 8821,
 'cut_events_3_3': 637,
 'cut_events_3_4': 663,
 'cut_events_4': 4874,
 'cut_events_5': 235,
 'cut_events_6': 637}

In [5]:
result['fields']

KeyError: 'fields'

```
['run',
 'GenJetAK8',
 'fixedGridRhoFastjetCentralChargedPileUp',
 'GenVtx',
 'SoftActivityJet',
 'fixedGridRhoFastjetCentral',
 'PuppiMET',
 'Tau',
 'TrigObj',
 'SoftActivityJetNjets10',
 'GenModel',
 'MET',
 'fixedGridRhoFastjetAll',
 'Electron',
 'HLTriggerFinalPath',
 'genTtbarId',
 'Jet',
 'GenJet',
 'LowPtElectron',
 'SoftActivityJetNjets5',
 'PV',
 'SoftActivityJetHT',
 'DeepMETResolutionTune',
 'Muon',
 'SubJet',
 'IsoTrack',
 'OtherPV',
 'HLTriggerFirstPath',
 'GenPart',
 'ChsMET',
 'boostedTau',
 'LHEPdfWeight',
 'DeepMETResponseTune',
 'GenIsolatedPhoton',
 'SoftActivityJetHT2',
 'Pileup',
 'HLT',
 'SubGenJetAK8',
 'GenVisTau',
 'TkMET',
 'fixedGridRhoFastjetCentralNeutral',
 'CaloMET',
 'PSWeight',
 'LHEScaleWeight',
 'luminosityBlock',
 'fixedGridRhoFastjetCentralCalo',
 'L1PreFiringWeight',
 'btagWeight',
 'L1Reco',
 'LHEWeight',
 'FatJet',
 'L1',
 'RawMET',
 'RawPuppiMET',
 'HTXS',
 'SoftActivityJetHT5',
 'FsrPhoton',
 'event',
 'SoftActivityJetHT10',
 'Generator',
 'L1simulation',
 'SV',
 'GenDressedLepton',
 'genWeight',
 'GenMET',
 'Flag',
 'SoftActivityJetNjets2',
 'Photon',
 'CorrT1METJet',
]

result['PV_fields'] :

'ndof',

 'x',
 
 'y',
 
 'z',
 
 'chi2',
 
 'score',
 
 'npvs',
 
 'npvsGood'